In [1]:
# %load proxypool.py


# In[1]:

from urllib.parse import urlencode
import requests
from bs4 import BeautifulSoup
from lxml import etree
import time
import re
from fake_useragent import UserAgent

import sys


import requests
from lxml import etree

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import multiprocessing as mp


# In[32]:

class proxypool:
    # free proxy 網站列表，缺點是每個網站要爬的方式可能是不同的，最主力來源是 www.gatherproxy.com
    # 這個list的順序不要亂換，因為針對不同的free proxy網站parser是寫好固定的爬法的
    proxy_html = ['http://free-proxy-list.net', 
                  'http://www.gatherproxy.com/proxylist/country/?c=', 
                  'https://www.us-proxy.org/',  
                  'http://www.proxylisty.com/country/Taiwan-ip-list']
    # 當給出proxy之前，會對proxy_test內網站做測試，前3個是隨便測的，最後一個是目標網站
    proxy_test = ['http://www.google.com.tw', 'http://www.google.co.jp', 'http://www.yahoo.com', 'http://findbiz.nat.gov.tw/fts/query/QueryBar/queryInit.do']
    
    # country_*的內容主要是給www.gatherproxy.com用，這些字串會塞在www.gatherproxy.com/proxylist/country/?c=後面
    # 以下針對不同區域整理起來，每個list放6個
    country_asia1 = ['Taiwan', 'Japan', 'Singapore', 'Korea','Republic%20of%20Korea', 'Hong%20Kong']
    country_asia2 = ['Israel', 'China', 'India', 'Iran', 'Pakistan']
    country_asia3 = ['Indonesia', 'Thailand', 'Vietnam', 'Philippines', 'Cambodia', 'Malaysia']
    country_asia4 = ['Iraq', 'Nepal', 'Mongolia', 'Bangladesh', 'Kazakhstan']
    asia = [country_asia1, country_asia2, country_asia3, country_asia4]
    
    country_europe1 = ['Russia', 'France', 'Germany', 'United%20Kingdom', 'Netherlands']
    country_europe2 = ['Denmark', 'Poland', 'Ukraine', 'Italy', 'Spain']
    country_europe3 = ['Turkey','Slovak%20Republic', 'Bulgaria', 'Czech%20Republic', 'Czechia', 'Hungary']
    country_europe4 = ['Romania', 'Serbia', 'Albania', 'Republic%20of%20Moldova', 'Bosnia%20and%20Herzegovina']
    europe = [country_europe1, country_europe2, country_europe3, country_europe4]
    
    country_north_america1 = ['United%20States', 'Canada']
    north_america = [country_north_america1]
    
    country_south_america1 = ['Brazil', 'Venezuela', 'Colombia', 'Argentina', 'Mexico', 'Ecuador']
    country_south_america2 = ['Ecuador', 'Chile', 'Peru', 'Paraguay']
    south_america = [country_south_america1, country_south_america2]
    
    country_oceania1 = ['Australia']
    oceania = [country_oceania1]
    
    country_africa1 = ['South%20Africa', 'Nigeria', 'Egypt', 'Kenya']
    africa = [country_africa1]
    
    country_group1 = country_asia1 + country_europe1 + country_europe2 + country_north_america1 + country_oceania1
    country_group2 = country_asia2 + country_europe3 + country_south_america1 + country_africa1
    country_group3 = country_asia3 + country_asia4 + country_europe4
    group = [country_group1, country_group2, country_group3]
    
    country_world = country_group1 + country_group2 + country_group3
    
    # 置在抓proxy時是先塞到proxy_set，以過濾重複的proxy
    # 再由proxy_set轉為proxy_list，並將之轉換成requests套件要用的dict，並提供random choice使用，總之最後丟出去的東西從這裡來
    # proxy_set_max主要針對
    proxy_list = list()
    proxy_set = set()
    proxy_set_max = 100
    #proxy_re = re.compile('[0-9]+(?:\.[0-9]+){3}:\d{2,4}')
    
    # 由於 www.gatherproxy.com需要用到點'下一頁'的按鈕，所以需要給定phantomjs的執行檔路徑
    def __init__(self, none_freq = 2, path_phantomjs = '/usr/local/Cellar/phantomjs/2.1.1/bin/phantomjs'):
        self.session = None
        self.response = None
        self.path_phantomjs = path_phantomjs
        self.none_freq = none_freq
        self.none_freq_cnt = 0
        self.asia_i = -1
        self.europe_i = -1
        self.north_america_i = -1
        self.south_america_i = -1
        self.group_i = -1
        self.oceania_i = -1
        self.africa_i = -1
        
    # 重新連線時先關掉舊session再開新的session
    def new_session(self):
        if self.session is not None:
            self.session.close()
        
        self.session = requests.Session()
        
    # 清除proxy_set, proxy_list
    def reset_proxy(self):
        del proxypool.proxy_set
        del proxy_list
        proxypool.proxy_set = set()
        proxypool.proxy_list = list()
       
    # 將proxy_set轉成proxy_list，proxy_list的內容(dict)才是要傳出去的
    def proxy_set_to_proxy_list(list):
        proxypool.proxy_list = [{'http':p} for p in proxypool.proxy_set]
    
    # 針對proxy_html[0]的網站爬蟲
    def get_proxy1(self):
        self.new_session()
        self.response = self.session.get(proxypool.proxy_html[0])
        selector = etree.HTML(self.response.content)
        trs = selector.xpath('//*[@id="proxylisttable"]/tbody/tr')
        for tr in trs:
            tds = tr.xpath('./td')
            ip = tds[0].text
            port = tds[1].text
            proxypool.proxy_set.add(ip+':'+port)

    # 針對proxy_html[1]的網站爬蟲
    def get_proxy2(self, country ='Taiwan'):
        #PhantomJs_executable_path='/usr/local/Cellar/phantomjs/2.1.1/bin/phantomjs'
        def clean_text(text):
            import re
            if text is None:
                text = ''
                return text
        
            text = text.encode('latin_1', errors='ignore').decode('utf8', errors='ignore')
            text = re.sub(r'[\t\n\r]', r'', text)
            return text
        # step 1. use PhantomJs to get .js rendered content
        browser = webdriver.PhantomJS(executable_path = self.path_phantomjs)
        browser.get(proxypool.proxy_html[1]+country)

        # step 2. click "Show Full List" button to generate full proxies list
        element = WebDriverWait(browser, 1).until(
            EC.presence_of_element_located((By.XPATH, '//div[@id="body"]/form/p/input[@type="submit" and @class="button"]')))
        element.click()
        
        # step 3. generate selector
        selector = etree.HTML(browser.page_source)

        # step 4. resolve how many pages
        pages = selector.xpath('//div[@id="body"]/form[@id="psbform"]/div[@class="pagenavi"]/a')

        # setp 5. resolve trs for first page
        trs = selector.xpath('//div[@class="proxy-list"]/table[@id="tblproxy"]/tbody/tr')  
        for tr in trs[2:]:
            tds = tr.xpath('./td')
            if len(tds) < 2:
                continue
            ip = "" if not tds[1].xpath('./text()') else tds[1].xpath('./text()')[0]
            port = "" if not tds[2].xpath('./text()') else tds[2].xpath('./text()')[0]
            proxypool.proxy_set.add(ip+':'+port)
        #print('page 0 done')

    
        # step 6. resolve trs for the rest pages
        for i, page in enumerate(pages, 2):
            # step 6-1. click nextpage's link
            element = WebDriverWait(browser, 1).until(
                EC.presence_of_element_located((By.XPATH, '//div[@id="body"]/form[@id="psbform"]/div[@class="pagenavi"]/a[@href="#{}"]'.format(i))))
            element.click()
            
            # step 6-2. resolve trs for ith page
            selector = etree.HTML(browser.page_source)
            trs = selector.xpath('//div[@class="proxy-list"]/table[@id="tblproxy"]/tbody/tr')
            for tr in trs[2:]:
                tds = tr.xpath('./td')
                if len(tds) < 2:
                    continue
                ip = "" if not tds[1].xpath('./text()') else tds[1].xpath('./text()')[0]
                port = "" if not tds[2].xpath('./text()') else tds[2].xpath('./text()')[0]
                proxypool.proxy_set.add(ip+':'+port)
                if len(proxypool.proxy_set) > proxypool.proxy_set_max:
                    return
            # 防反爬蟲，這個網站太頻繁抓賄被擋
            time.sleep(5)
            #print('page', i,'done')
    
    # 針對proxy_html[2]的網站爬蟲
    def get_proxy3(self):
        self.new_session()
        self.response = self.session.get(proxypool.proxy_html[2])
        selector = etree.HTML(self.response.content)
        trs = selector.xpath('//*[@id="proxylisttable"]/tbody/tr')
        for tr in trs:
            tds = tr.xpath('./td')
            ip = tds[0].text
            port = tds[1].text
            proxypool.proxy_set.add(ip+':'+port)

    # 針對proxy_html[3]的網站爬蟲
    def get_proxy4(self):
        self.new_session()
        self.response = self.session.get(proxypool.proxy_html[3])
        selector = etree.HTML(self.response.content)
        trs = selector.xpath('//*[@id="content"]/table[1]/tr')
        for tr in trs[2:-2]:
            tds = tr.xpath('./td')
            if tds:
                ip = tds[0].text
                port = tds[1].xpath('./a')[0].text
                #print(ip, port)
                proxypool.proxy_set.add(ip+':'+port)

    # proxypool.asia/eurpoe/oceania/north_america/south_america/africa 都是2維list
    # 每次只從中挑一個子list出來，目的是防範對www.gatherproxy.com發出太多request
    # 例：asia = [country_asia1, country_asia2, country_asia3, country_asia4]
    # 則每次self.pick_country(area = 'asia')會依序return country_asia1, country_asia2 ...
    def pick_country(self, area = 'asia'):
        if area == 'asia':
            self.asia_i += 1
            return proxypool.asia[self.asia_i % len(proxypool.asia)]
        elif area == 'europe':
            self.europe_i += 1
            return proxypool.europe[self.europe_i % len(proxypool.europe)]
        elif area == 'oceania':
            self.oceania_i += 1
            return proxypool.oceania[self.oceania_i % len(proxypool.oceania)]
        elif area == 'north_america':
            self.north_america_i += 1
            return proxypool.north_america[self.north_america_i % len(proxypool.north_america)]
        elif area == 'south_america':
            self.south_america_i += 1
            return proxypool.south_america[self.south_america_i % len(proxypool.south_america)]
        elif area == 'africa':
            self.africa_i += 1
            return proxypool.africa[self.africa_i % len(proxypool.africa)]
        elif area == 'world':
            return proxypool.country_world
        elif area == 'group':
            self.group_i += 1
            return proxypool.group[self.group_i % len(proxypool.group)]
        else:
            return list()
    # 從pick_country return的list依序去 www.gatherproxy.com 爬proxy
    def get_proxy2_by_pick_country(self, area = 'asia'):
        for c in self.pick_country(area):
            self.get_proxy2(country = c)        
                
    # 以下都是針對特定區域去抓proxy
    def world_proxy(self):
        self.get_proxy1()
        self.get_proxy2_by_pick_country('world')
        self.get_proxy3()
        self.get_proxy4()
    
    def eu_proxy(self):
        self.get_proxy2_by_pick_country('europe')

    def na_proxy(self):
        self.get_proxy2_by_pick_country('north_america')
        self.get_proxy3()
        
    def taiwan_proxy(self):
        self.reset_proxy()
        self.get_proxy2(country ='Taiwan')
        self.get_proxy4()
    
    def asia_proxy(self):
        self.get_proxy2_by_pick_country('asia')
        self.get_proxy4()

    def group_proxy(self):
        self.get_proxy2_by_pick_country('group')        
        
    # 舊的方針是一口氣抓很多proxy，每個proxy都檢驗，最後將合格的放到proxy_set，需要的時候再挑出一個
    # 但舊的方針不好，因為proxy有時效性，新的方針為要挑出一個proxy前先檢驗
    # filter_proxy會對所有的proxy做檢驗，暫時沒用到
    def filter_proxy(self):
        bad_proxy_set = set()
        for p in proxypool.proxy_set:
            connection_score = list()
            for t in proxypool.proxy_test:
                self.new_session()
                
                if not check_this_proxy(p):
                    bad_proxy_set.add(p)
        else:
            proxypool.proxy_set = proxypool.proxy_set - bad_proxy_set
            self.proxy_set_to_proxy_list()
                    
    # 針對proxy做檢驗，會針對cpu核心數平行檢驗
    # 檢查成功return True，檢查失敗return False
    def check_this_proxy(self, p):
        print('checking', p)
        # 針對某個porxy對某個url檢查response status是否為200，例外發生時要關掉session之後再重開
        def check_proxy_on_url(p, url):
            sess = requests.Session()
            try:
                self.response = sess.get(t, timeout=30, proxies=p)
                #print('checking', p,'@', t,'get response.status_code', self.response)
                sess.close()
                Q.put((p, self.response.status_code == 200))
            except Exception as err:
                #print('checking', p,'@', t, 'Exception happened:', err)
                if sess:
                    sess.close()
                Q.put((p, False))

        # 1. 取得cpu數量
        cpus = mp.cpu_count()
        #print('cpus:', cpus)
        
        # 2-1. 多核心時平行檢查proxy跟url的連線狀態
        if cpus > 1:
            rounds = len(proxypool.proxy_test) // cpus
            Q = mp.JoinableQueue()
            for r in range(rounds):
                plist = list()
                for t in proxypool.proxy_test[cpus*r : cpus*(r+1)]:
                    pi = mp.Process(target=check_proxy_on_url, args=(p, t))
                    plist.append( pi )
                    pi.start()
        
                Q.join()
                for pi in plist:
                    pi.join()
            
                for pi in plist:
                    pi.terminate()
                    del pi
                else:
                    del plist
                
                while not Q.empty():
                    check = Q.get()
                    #print('Q.get():', check)
                    if not check[1]:
                        print('check fail')
                        return False
            else:
                print('check pass')
                return True
        # 2-2. 單核心時循序檢查proxy跟url的連線狀態
        else:
            for t in proxypool.proxy_test:
                sess = requests.Session()
                valid_proxy = True
                try:
                    self.response = sess.get(t, timeout=30, proxies=p)
                    #print('checking', p,'@', t,'get response.status_code', self.response)
                    sess.close()
                    valid_proxy = self.response.status_code == 200
                except Exception as err:
                    #print('checking', p,'@', t, 'Exception happened:', err)
                    if sess:
                        sess.close()
                    valid_proxy = False
                
                if not valid_proxy:
                    print('check fail')
                    return False
            else:
                print('check pass')
                return True

    # 不斷隨機從proxy_list挑出一個，並檢查，直到找到一個proxy是通過檢驗的才return出去
    # 若檢查的過程中proxy_set空了，就return None，代表不用proxy，用本身的ip去連網站
    # 若proxy_set空掉，需要使用者自行呼叫get_proxy()或其他爬proxy的method
    def random_choice_one_proxy(self):
        import random
        self.proxy_set_to_proxy_list()
        p = random.choice(self.proxy_list)
        del self.proxy_list[self.proxy_list.index(p)]
        self.proxy_set.remove( list(p.values())[0] )
        print('while')
        while not self.check_this_proxy(p):
            #print('in while')
            if len(proxypool.proxy_set) == 0:
                p = {'http':None}
                break
                
            p = random.choice(self.proxy_list)
            del self.proxy_list[self.proxy_list.index(p)]
            self.proxy_set.remove( list(p.values())[0] )
        return p
    
    # 每N次隨機proxy就有一次是None(用自己的ip連網站)，N取決於self.none_freq
    def random_choice_one_proxy_with_none_freq(self):
        self.none_freq_cnt += 1
        if self.none_freq_cnt % self.none_freq == 0:
            self.none_freq_cnt = 0
            return None
        else:
            return self.random_choice_one_proxy()

In [2]:
p = proxypool(none_freq = 5)
proxypool.proxy_set_max = 50
p.proxy_set_max

50

In [3]:
p.asia_proxy()

In [4]:
p.proxy_set

{'1.160.146.107:80',
 '1.160.146.75:80',
 '1.160.151.33:80',
 '1.160.155.38:80',
 '1.165.162.162:80',
 '1.165.182.158:80',
 '1.165.190.235:80',
 '1.172.56.201:8080',
 '1.172.57.143:8080',
 '1.172.59.92:8080',
 '1.173.125.51:3128',
 '1.173.27.226:3128',
 '1.174.126.81:3128',
 '1.175.107.112:3128',
 '1.175.111.129:3128',
 '1.175.134.72:3128',
 '1.175.218.35:3128',
 '1.225.109.161:3128',
 '1.34.126.189:80',
 '1.34.220.124:80',
 '1.34.223.21:80',
 '103.15.187.116:3128',
 '103.15.187.116:8000',
 '103.218.240.120:8118',
 '103.234.220.187:80',
 '103.253.146.48:8080',
 '103.37.95.110:81',
 '103.73.67.33:10000',
 '103.73.67.33:3128',
 '106.104.121.187:3128',
 '106.104.65.36:80',
 '110.135.245.99:3128',
 '111.171.219.110:3128',
 '111.249.112.100:80',
 '112.140.184.136:3128',
 '112.140.184.139:3129',
 '112.216.16.250:3128',
 '113.252.236.56:8080',
 '113.253.113.90:8380',
 '113.254.114.24:80',
 '113.254.97.68:80',
 '113.254.99.172:8080',
 '114.25.77.178:53281',
 '114.44.143.252:80',
 '114.44.191.6

In [5]:
# 這時候還沒轉成proxy_list
p.proxy_list

[]

In [6]:
# 需手動呼叫 p.proxy_set_to_proxy_list() 才會轉成list
p.proxy_set_to_proxy_list()
p.proxy_list

[{'http': '42.61.52.115:3128'},
 {'http': '220.132.30.63:3128'},
 {'http': '210.242.163.176:8080'},
 {'http': '112.216.16.250:3128'},
 {'http': '165.225.96.74:10000'},
 {'http': '112.140.184.136:3128'},
 {'http': '1.34.126.189:80'},
 {'http': '165.225.100.80:10000'},
 {'http': '210.242.163.175:8080'},
 {'http': '146.185.205.194:8085'},
 {'http': '112.140.184.139:3129'},
 {'http': '121.140.126.250:3128'},
 {'http': '103.234.220.187:80'},
 {'http': '1.34.220.124:80'},
 {'http': '114.25.77.178:53281'},
 {'http': '202.157.182.141:3128'},
 {'http': '211.245.31.94:80'},
 {'http': '59.115.91.217:3128'},
 {'http': '165.225.102.65:10000'},
 {'http': '113.254.114.24:80'},
 {'http': '1.174.126.81:3128'},
 {'http': '1.160.155.38:80'},
 {'http': '128.199.77.93:8000'},
 {'http': '165.225.96.71:10000'},
 {'http': '218.161.47.231:3128'},
 {'http': '128.199.193.44:3128'},
 {'http': '13.78.125.167:8080'},
 {'http': '139.59.245.95:3128'},
 {'http': '27.120.85.178:8088'},
 {'http': '47.91.159.195:8080'},


In [7]:
p.random_choice_one_proxy_with_none_freq()

while
checking {'http': '58.124.172.52:80'}
check fail
checking {'http': '124.6.10.162:80'}
check pass


{'http': '124.6.10.162:80'}

In [8]:
p.random_choice_one_proxy_with_none_freq()

while
checking {'http': '165.225.102.64:10000'}
check fail
checking {'http': '61.220.204.25:3128'}
check fail
checking {'http': '202.73.51.102:8128'}
check fail
checking {'http': '220.80.174.36:80'}
check fail
checking {'http': '106.104.65.36:80'}
check fail
checking {'http': '139.59.245.95:3128'}
check pass


{'http': '139.59.245.95:3128'}

In [9]:
p.random_choice_one_proxy_with_none_freq()

while
checking {'http': '180.44.204.77:8080'}
check fail
checking {'http': '175.194.88.51:3128'}
check fail
checking {'http': '58.181.39.182:80'}
check pass


{'http': '58.181.39.182:80'}

In [10]:
p.random_choice_one_proxy_with_none_freq()

while
checking {'http': '103.73.67.33:3128'}
check fail
checking {'http': '218.255.79.154:3128'}
check pass


{'http': '218.255.79.154:3128'}

In [11]:
# 第5次，回傳一個None
p.random_choice_one_proxy_with_none_freq() is None

True